In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from pandas import DataFrame
from collections import Counter
%matplotlib inline
import os
import seaborn as sns
from glob import glob

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, MaxPooling2D, AveragePooling2D, Conv2D, Dropout, BatchNormalization, GlobalMaxPooling2D, Flatten
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8165737456751619958
]


In [14]:
print(tf.__version__)

2.4.1


In [4]:
img_input = tf.keras.Input(shape=(128, 128, 3))
x = tf.keras.layers.Conv2D(32, (5, 5), padding='SAME', activation='relu')(img_input)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Flatten()(x)
# x=tf.keras.layers.Dense(128,activation='relu')(x)
# x=tf.keras.layers.Dense(1,activation='sigmoid')(x)

crowd_prediction = tf.keras.layers.Dense(1, activation='sigmoid', name='crowd')(x)
coeff_prediction = tf.keras.layers.Dense(1, name='coeff')(x)

model = tf.keras.Model(img_input, [crowd_prediction, coeff_prediction])
sgd = tf.keras.optimizers.SGD(learning_rate=0.01)

model.compile(optimizer=sgd,
              loss=['binary_crossentropy', 'mae'],
              metrics=['accuracy', 'mae'])

In [5]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 128, 128, 32) 2432        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 62, 62, 64)   18496       max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [6]:
train_class = glob('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/train/*')
print(f'Traning classes count: {len(train_class)}')
val_class = glob('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/val/*')
print(f'Validation classes count: {len(val_class)}')
test_class = glob('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/test/*')
print(f'Testing classes count: {len(test_class)}')

Traning classes count: 2
Validation classes count: 2
Testing classes count: 2


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
BATCH_SIZE = 64

training_set = train_datagen.flow_from_directory('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/train', 
                                                 target_size=(128,128),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary')

validation_set = val_datagen.flow_from_directory('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/val', 
                                                 target_size=(128,128),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary')

testing_set = test_datagen.flow_from_directory('D:/D/RenaNozawa/Kuliah/ITS/TA/dataset/Newfolder/test', 
                                               target_size=(128,128),
                                               batch_size=BATCH_SIZE,
                                               class_mode='binary', 
                                               shuffle=False)

test_steps_per_epoch = np.math.ceil(testing_set.samples / testing_set.batch_size)

Found 24201 images belonging to 2 classes.
Found 2846 images belonging to 2 classes.
Found 6635 images belonging to 2 classes.


In [12]:
history = model.fit_generator(generator=training_set, 
                              validation_data=validation_set, 
                              epochs=10, 
                              steps_per_epoch=24201/BATCH_SIZE, 
                              validation_steps=2846/BATCH_SIZE
                              )

C:\Users\timot\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
 33/378 [=>............................] - ETA: 34:28 - loss: 1.4801 - crowd_loss: 0.6901 - coeff_loss: 0.7900 - crowd_accuracy: 0.5380 - crowd_mae: 0.4975 - coeff_accuracy: 0.4844 - coeff_mae: 0.7900

KeyboardInterrupt: 